### The aim of the project and the story of dataset

In this study, it is aimed to test the interface screens of the mobile application called 'Cloud Library', which is a book sharing platform, on users and to create a new interface system as a result of this test.'Sign-up, Recommendation and Notification Settings' interfaces have been tested on users. 

The user testing process was made in Turkey and the Netherlands. The study was conducted on 70 people in total, 35 from each country.

    .User_group_1 -> Survey in Turkey
    .User_group_2 -> Survey in Netherlands

In [1]:
import pandas as pd
from scipy.stats import shapiro, levene
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [2]:
user_group_1 = pd.read_csv(r'/Users/firatsoydinc/Desktop/Master /Block - B/Smart Interface/A:B User Testing /user_group_1.csv')
user_group_2 = pd.read_csv(r'/Users/firatsoydinc/Desktop/Master /Block - B/Smart Interface/A:B User Testing /user_group_2.csv')


In [3]:
user_group_1.rename({'Zaman damgası':'Time','Yaşınız nedir?':'Age','Eğitim durumunuz nedir?':'Education_Level','Cinsiyetiniz nedir? ':'Gender',
                     'Bir siteye kaydolurken "Google" veya " Apple" ile kaydolma seçeceği olduğunda bunları kullanmayı tercih eder misiniz?':'Singup_Preferences',
                     'Bildirim arayüzünün kullanıcıya göre ayarlanabilir olması sizin için önemli midir? (Belirli gün veya saatlerde bildirim göndermemesi gibi)':'Notification_Preferences',
                     'Kitap öneri ekranında bulunan kitapların sadece isim bilgisinin olmasını mı yoksa kitaba ait ön bilgilerin olmasını mı tercih edersiniz?':'Recommendation_Preferences'},axis=1,inplace=True)
user_group_1["Education_Level"].replace({'Lise':'Highschool',"Üniversite": "Bachelor", "Yüksek Lisans": "Master",'Diğer':'Other'}, inplace=True)
user_group_1["Gender"].replace({'Kadın':'Female',"Erkek": "Male", "Diğer": "Other"}, inplace=True)
user_group_1["Singup_Preferences"].replace({'Evet':'Yes',"Hayır": "No"}, inplace=True)
user_group_1["Notification_Preferences"].replace({'Evet':'Yes',"Hayır": "No"}, inplace=True)
user_group_1["Recommendation_Preferences"].replace({'Sadece kitabın fotoğrafları':'Only the name of the book',"Kitaba ait ön bilgiler": "Preliminary information of the book"}, inplace=True)
user_group_1["Age"].replace({'35-45':'35-44'}, inplace=True)



Since the study was conducted using google surveys, it is necessary to organize the dataset. In addition, since the results of the survey conducted in Turkey are in Turkish, the answers have been translated into English. In addition, since the variable names consisted of the questions in the questionnaire, the renaming was done.

In [4]:
user_group_2.rename({'Zaman damgası':'Time','What is your age? ':'Age','What is your education level? ':'Education_Level','What is your gender? ':'Gender',
                     'When registering for a site, would you prefer to use them when there is an option to register with a "Google" and "Apple" account?':'Singup_Preferences',
                     'Is it important for you that the notification interface is user-adjustable? (Like not sending notifications on certain days or hours)':'Notification_Preferences',
                     'Do you prefer the books on the book recommendation screen to have only the name information or to have the preliminary information about the book?':'Recommendation_Preferences'},axis=1,inplace=True)

#Since there was no response given by a user to the recommendation system question, it was filled with the mod of variable
user_group_2['Recommendation_Preferences'].fillna('Preliminary information of the book')

#Also, since the time variable did not create a value for the study, it was removed from the data set.
user_group_1.drop(['Time'],axis=1,inplace=True)
user_group_2.drop(['Time'],axis=1,inplace=True)



In [5]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """
    grab_col_names for given dataframe

    :param dataframe:
    :param cat_th:
    :param car_th:
    :return:
    """

    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]

    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]

    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]

    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

#The num_but_cat variable is included in the cat_cols variable. Therefore, adding cat_cols, 
#num_cols and cat_but_car values will be sufficient for the total number of variables.

    return cat_cols, cat_but_car, num_cols, num_but_cat


In [6]:
cat_cols_1, cat_but_car_1, num_cols_1, num_but_cat_1 = grab_col_names(user_group_1)

Observations: 35
Variables: 6
cat_cols: 6
num_cols: 0
cat_but_car: 0
num_but_cat: 0


In [7]:
cat_cols_2, cat_but_car_2, num_cols_2, num_but_cat_2 = grab_col_names(user_group_2)

Observations: 35
Variables: 6
cat_cols: 6
num_cols: 0
cat_but_car: 0
num_but_cat: 0


In order to convert the categorical variables in the data set to numerical variables, the one-hot encoding process is applied to the data set.

In [8]:
def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

user_group_1 = one_hot_encoder(user_group_1, cat_cols_1, drop_first=True)
user_group_2 = one_hot_encoder(user_group_2, cat_cols_2, drop_first=True)

### A/B Test
A/B test is performed to check whether there is a statistically significant difference between two independent data sets. There are 2 different A/B test methods. One of them is parametric and the other is nonparametric. In order to perform a parametric test, the data set must provide 3 assumptions. If even one of these assumptions is not met, non-parametric testing is applied.

Assumptions: 
    
    Number of observations must be at least 30  (n>30)
    
    Variables must be normal distributed 
    
    Variance homogeneity must be assigned by variables

#### First Assumption n>30

In [9]:
user_group_1.shape

(35, 7)

In [10]:
user_group_2.shape

(35, 9)

It includes more than 30 observations in both groups.

#### Second Assumption (Normal Distribution)

    H1: Variables are normally distributed.
    H0: Variables are not normally distributed.
    
If p_value is less than 0.05 h1 is reject, else fail to reject 

In [11]:
for each in user_group_1.columns:
    test_stat, pvalue = shapiro(user_group_1[each])
    print('Test Stat of ', each ,'= %.4f, p-value = %.4f' % (test_stat, pvalue))


Test Stat of  Age_25-34 = 0.3718, p-value = 0.0000
Test Stat of  Age_35-44 = 0.2500, p-value = 0.0000
Test Stat of  Education_Level_Master = 0.5458, p-value = 0.0000
Test Stat of  Gender_Male = 0.6373, p-value = 0.0000
Test Stat of  Singup_Preferences_Yes = 0.5855, p-value = 0.0000
Test Stat of  Notification_Preferences_Yes = 0.4575, p-value = 0.0000
Test Stat of  Recommendation_Preferences_Preliminary information of the book = 0.5673, p-value = 0.0000


In [12]:

for each in user_group_2.columns:
    test_stat, pvalue = shapiro(user_group_2[each])
    print('Test Stat of ', each ,'= %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat of  Age_25-34 = 0.5458, p-value = 0.0000
Test Stat of  Age_35-44 = 0.1615, p-value = 0.0000
Test Stat of  Age_45-54 = 0.1615, p-value = 0.0000
Test Stat of  Education_Level_Highschool = 0.4914, p-value = 0.0000
Test Stat of  Education_Level_Master = 0.4914, p-value = 0.0000
Test Stat of  Gender_Male = 0.6229, p-value = 0.0000
Test Stat of  Singup_Preferences_Yes = 0.5458, p-value = 0.0000
Test Stat of  Notification_Preferences_Yes = 0.4575, p-value = 0.0000
Test Stat of  Recommendation_Preferences_Preliminary information of the book = 0.4914, p-value = 0.0000


All variables in the both data set provide the assumption of normality.

#### Third Assumption (Variance Homogenity)

    H1: Variances of Variables Are Homogeneously Distributed
    H0: Not homogeneously distributed
   
If p_value is less than 0.05 h1 is reject, else fail to reject 

In [13]:
for each in user_group_1.columns:
    test_stat, pvalue = shapiro(user_group_1[each])
    print('Test Stat of ', each ,'= %.4f, p-value = %.4f' % (test_stat, pvalue))


Test Stat of  Age_25-34 = 0.3718, p-value = 0.0000
Test Stat of  Age_35-44 = 0.2500, p-value = 0.0000
Test Stat of  Education_Level_Master = 0.5458, p-value = 0.0000
Test Stat of  Gender_Male = 0.6373, p-value = 0.0000
Test Stat of  Singup_Preferences_Yes = 0.5855, p-value = 0.0000
Test Stat of  Notification_Preferences_Yes = 0.4575, p-value = 0.0000
Test Stat of  Recommendation_Preferences_Preliminary information of the book = 0.5673, p-value = 0.0000


In [14]:
for each in user_group_2.columns:
    test_stat, pvalue = shapiro(user_group_2[each])
    print('Test Stat of ', each ,'= %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat of  Age_25-34 = 0.5458, p-value = 0.0000
Test Stat of  Age_35-44 = 0.1615, p-value = 0.0000
Test Stat of  Age_45-54 = 0.1615, p-value = 0.0000
Test Stat of  Education_Level_Highschool = 0.4914, p-value = 0.0000
Test Stat of  Education_Level_Master = 0.4914, p-value = 0.0000
Test Stat of  Gender_Male = 0.6229, p-value = 0.0000
Test Stat of  Singup_Preferences_Yes = 0.5458, p-value = 0.0000
Test Stat of  Notification_Preferences_Yes = 0.4575, p-value = 0.0000
Test Stat of  Recommendation_Preferences_Preliminary information of the book = 0.4914, p-value = 0.0000


All variables in the both data set provide the assumption of variance homogenity.

#### 1. Sign - Up Interface

    H1: People prefer to enroll to an app with using sercive provider accounts like Google, Apple and so on.
    H0: People don't prefer to enroll to an app with using sercive provider accounts like Google, Apple and so on.

In [15]:
test_stat, pvalue = proportions_ztest(count=[((user_group_1['Singup_Preferences_Yes'].value_counts()))[0], (user_group_2['Singup_Preferences_Yes'].value_counts())[1]],
                                      nobs=[user_group_1['Singup_Preferences_Yes'].count(), user_group_2['Singup_Preferences_Yes'].count()])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -3.5916, p-value = 0.0003


In [16]:
ratio_of_signup_1 = (user_group_1['Singup_Preferences_Yes'].value_counts())[0]/user_group_1['Singup_Preferences_Yes'].count()
print('The ratio of people in user_group_1 who not prefer to enroll with sercive provider accounts is ', round(ratio_of_signup_1,4) )

The ratio of people in user_group_1 who not prefer to enroll with sercive provider accounts is  0.3143


In [17]:
ratio_of_signup_2 = (user_group_2['Singup_Preferences_Yes'].value_counts())[1]/user_group_1['Singup_Preferences_Yes'].count()
print('The ratio of people in user_group_2 who prefer to enroll with sercive provider accounts is ', round(ratio_of_signup_2,4) )

The ratio of people in user_group_2 who prefer to enroll with sercive provider accounts is  0.7429


The p-value is less than 0.05 so the H1 hypothesis fails to reject. In addition, since the rate of those who prefer to use service providers to register for the application is higher than those who do not, registration with service providers has been used as an interface screen.

#### 2. Recommendation System Screen 

    H1: People want to see more information about the product in their recommendation system.
    H0: People don't want to see more information about the product in their recommendation system.


In [18]:
test_stat, pvalue = proportions_ztest(count=[((user_group_1['Recommendation_Preferences_Preliminary information of the book'].value_counts()))[0], (user_group_2['Recommendation_Preferences_Preliminary information of the book'].value_counts())[1]],
                                      nobs=[user_group_1['Recommendation_Preferences_Preliminary information of the book'].count(), user_group_2['Recommendation_Preferences_Preliminary information of the book'].count()])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -4.3187, p-value = 0.0000


In [19]:
ratio_of_recommendation_1 = (user_group_1['Recommendation_Preferences_Preliminary information of the book'].value_counts())[0]/user_group_1['Recommendation_Preferences_Preliminary information of the book'].count()
print('The ratio of people in user_group_1 who not prefer to see more information on their recommendation screen is ', round(ratio_of_recommendation_1,4) )

The ratio of people in user_group_1 who not prefer to see more information on their recommendation screen is  0.2857


In [20]:
ratio_of_recommendation_2 = (user_group_2['Recommendation_Preferences_Preliminary information of the book'].value_counts())[1]/user_group_1['Recommendation_Preferences_Preliminary information of the book'].count()
print('The ratio of people in user_group_2 who not prefer to see more information on their recommendation screen is ', round(ratio_of_recommendation_2,4) )


The ratio of people in user_group_2 who not prefer to see more information on their recommendation screen is  0.8


The p-value is less than 0.05 so the H1 hypothesis fails to reject. In addition, since the rate of those who prefer to see more information about the product on the recommendation system screens is higher than those who do not, it has been decided to use an interface with more information in the recommendation system interface.

#### 3. Notification Settings    
    
    H1: People want to adjust notifications in the app.
    H0: People don't want to adjust notifications in the app

In [21]:
test_stat, pvalue = proportions_ztest(count=[((user_group_1['Notification_Preferences_Yes'].value_counts()))[0], (user_group_2['Notification_Preferences_Yes'].value_counts())[1]],
                                      nobs=[user_group_1['Notification_Preferences_Yes'].count(), user_group_2['Notification_Preferences_Yes'].count()])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))



Test Stat = -5.4981, p-value = 0.0000


In [22]:
ratio_of_notification_1 = (user_group_1['Notification_Preferences_Yes'].value_counts())[0]/user_group_1['Notification_Preferences_Yes'].count()
print('The ratio of people in user_group_1 who not want to adjust notifications ', round(ratio_of_notification_1,4) )


The ratio of people in user_group_1 who not want to adjust notifications  0.1714


In [23]:
ratio_of_notification_2 = (user_group_2['Notification_Preferences_Yes'].value_counts())[1]/user_group_1['Notification_Preferences_Yes'].count()
print('The ratio of people in user_group_2 who want to adjust notifications ', round(ratio_of_notification_2,4) )


The ratio of people in user_group_2 who want to adjust notifications  0.8286


The p-value is less than 0.05 so the H1 hypothesis fails to reject. In addition, since the rate of those who prefer to adjust notifications is higher than those who do not, it has been decided to use an interface with adjustable notifications.